# --- Célula 1: Importações e Criação da SparkSession ---

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count

In [2]:
spark = SparkSession.builder \
    .appName("CSVDataLoading") \
    .getOrCreate()

print("SparkSession criada com sucesso!")
print(f"Versão do Spark: {spark.version}")
print(f"URL do Master: {spark.sparkContext.master}")

SparkSession criada com sucesso!
Versão do Spark: 3.5.0
URL do Master: local[*]


In [3]:
sample_products_path = "/home/jovyan/work/datasample/acidentes2007.csv"

print(f"\nCarregando dados de: {sample_products_path}")

try:
    df_products_sample = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv(sample_products_path)

    print("Schema do DataFrame de produtos (amostra):")
    df_products_sample.printSchema()

    print("\nConteúdo do DataFrame de produtos (amostra):")
    df_products_sample.show(5) # Mostra as 5 primeiras linhas

    print(f"Total de linhas no DataFrame de produtos (amostra): {df_products_sample.count()}")

except Exception as e:
    print(f"Erro ao carregar o CSV de amostra: {e}")


Carregando dados de: /home/jovyan/work/datasample/acidentes2007.csv
Schema do DataFrame de produtos (amostra):
root
 |-- id: integer (nullable = true)
 |-- pesid: integer (nullable = true)
 |-- data_inversa: string (nullable = true)
 |-- dia_semana: string (nullable = true)
 |-- horario: timestamp (nullable = true)
 |-- uf: string (nullable = true)
 |-- br: string (nullable = true)
 |-- km: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- causa_acidente: string (nullable = true)
 |-- tipo_acidente: string (nullable = true)
 |-- classificacao_acidente: string (nullable = true)
 |-- fase_dia: string (nullable = true)
 |-- sentido_via: string (nullable = true)
 |-- condicao_metereologica: string (nullable = true)
 |-- tipo_pista: string (nullable = true)
 |-- tracado_via: string (nullable = true)
 |-- uso_solo: string (nullable = true)
 |-- id_veiculo: string (nullable = true)
 |-- tipo_veiculo: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- ano_